In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("../../Data/Kaggle/StoreSales/new_train.csv")
test = pd.read_csv("../../Data/Kaggle/StoreSales/new_test.csv")

In [3]:
combined = pd.concat([train, test], ignore_index=True)

In [4]:
combined = pd.get_dummies(combined)

In [5]:
first_range = (0, 3000887)
second_range = (3000888, combined['id'].max())

In [6]:
first_range = (0, 3000887)
second_range = (3000888, combined['id'].max())

In [7]:
train = combined[(combined['id'] >= first_range[0]) & (combined['id'] <= first_range[1])]
test = combined[(combined['id'] >= second_range[0]) & (combined['id'] <= second_range[1])]

In [8]:
train.head()

,id,store_nbr,sales,onpromotion,dcoilwtico,cluster,day_of_week,month,year,date_2013-01-01,...,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua,typestores_A,typestores_B,typestores_C,typestores_D,typestores_E
0,0,1,0.0,0,93.14,13,2,1,2013,True,...,False,True,False,False,False,False,False,False,True,False
1,1,1,0.0,0,93.14,13,2,1,2013,True,...,False,True,False,False,False,False,False,False,True,False
2,2,1,0.0,0,93.14,13,2,1,2013,True,...,False,True,False,False,False,False,False,False,True,False
3,3,1,0.0,0,93.14,13,2,1,2013,True,...,False,True,False,False,False,False,False,False,True,False
4,4,1,0.0,0,93.14,13,2,1,2013,True,...,False,True,False,False,False,False,False,False,True,False


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000888 entries, 0 to 3000887
Columns: 1791 entries, id to typestores_E
dtypes: bool(1782), float64(2), int64(7)
memory usage: 5.2 GB


In [10]:
test.head()

,id,store_nbr,sales,onpromotion,dcoilwtico,cluster,day_of_week,month,year,date_2013-01-01,...,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua,typestores_A,typestores_B,typestores_C,typestores_D,typestores_E
3000888,3000888,1,NaN,0,46.8,13,3,8,2017,False,...,False,True,False,False,False,False,False,False,True,False
3000889,3000889,1,NaN,0,46.8,13,3,8,2017,False,...,False,True,False,False,False,False,False,False,True,False
3000890,3000890,1,NaN,2,46.8,13,3,8,2017,False,...,False,True,False,False,False,False,False,False,True,False
3000891,3000891,1,NaN,20,46.8,13,3,8,2017,False,...,False,True,False,False,False,False,False,False,True,False
3000892,3000892,1,NaN,0,46.8,13,3,8,2017,False,...,False,True,False,False,False,False,False,False,True,False


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28512 entries, 3000888 to 3029399
Columns: 1791 entries, id to typestores_E
dtypes: bool(1782), float64(2), int64(7)
memory usage: 50.6 MB


In [12]:
X = train.drop(['id', 'sales'], axis=1)
y = train['sales']

In [13]:
test_data = test.drop(['id', 'sales'], axis=1)

In [14]:
test_data.head()

,store_nbr,onpromotion,dcoilwtico,cluster,day_of_week,month,year,date_2013-01-01,date_2013-01-02,date_2013-01-03,...,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua,typestores_A,typestores_B,typestores_C,typestores_D,typestores_E
3000888,1,0,46.8,13,3,8,2017,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3000889,1,0,46.8,13,3,8,2017,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3000890,1,2,46.8,13,3,8,2017,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3000891,1,20,46.8,13,3,8,2017,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3000892,1,0,46.8,13,3,8,2017,False,False,False,...,False,True,False,False,False,False,False,False,True,False


In [15]:
batch_size = 10000

In [16]:
num_batches = len(X) // batch_size + 1

In [17]:
model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

In [18]:
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    X_batch = X.iloc[start_idx:end_idx]
    y_batch = y.iloc[start_idx:end_idx]

    model.fit(X_batch, y_batch)

    print(f"Batch {i + 1}/{num_batches} completed")

Batch 1/301 completed
Batch 2/301 completed
Batch 3/301 completed
Batch 4/301 completed
Batch 5/301 completed
Batch 6/301 completed
Batch 7/301 completed
Batch 8/301 completed
Batch 9/301 completed
Batch 10/301 completed
Batch 11/301 completed
Batch 12/301 completed
Batch 13/301 completed
Batch 14/301 completed
Batch 15/301 completed
Batch 16/301 completed
Batch 17/301 completed
Batch 18/301 completed
Batch 19/301 completed
Batch 20/301 completed
Batch 21/301 completed
Batch 22/301 completed
Batch 23/301 completed
Batch 24/301 completed
Batch 25/301 completed
Batch 26/301 completed
Batch 27/301 completed
Batch 28/301 completed
Batch 29/301 completed
Batch 30/301 completed
Batch 31/301 completed
Batch 32/301 completed
Batch 33/301 completed
Batch 34/301 completed
Batch 35/301 completed
Batch 36/301 completed
Batch 37/301 completed
Batch 38/301 completed
Batch 39/301 completed
Batch 40/301 completed
Batch 41/301 completed
Batch 42/301 completed
Batch 43/301 completed
Batch 44/301 complet

In [19]:
predictions = model.predict(test_data)

In [20]:
submission = pd.DataFrame({'id': test['id'], 'sales': predictions})

In [21]:
submission['sales'] = submission['sales'].apply(lambda x: max(0, x))

In [22]:
submission.to_csv('../../Data/Kaggle/StoreSales/submission.csv', index=False)